# Differentiable Quantum Architecture Search

## Overview

This tutorial demonstrates how to utilize the advanced computational features provided by TensorCircuit such as ``jit`` and ``vmap`` to super efficiently simulate the differentiable quantum architecture search (DQAS) algorithm, where an ensemble of quantum circuits with different structures can be compiled to simulate at the same time.

[WIP note]

## Setup

In [1]:
import numpy as np
import tensorcircuit as tc
import tensorflow as tf

In [2]:
K = tc.set_backend("tensorflow")
ctype, rtype = tc.set_dtype("complex128")

## Problem Description

The task is to find the state preparation circuit for GHZ state $\vert \text{GHZ}_N\rangle = \frac{1}{\sqrt{2}}\left(\vert 0^N\rangle +\vert 1^N\rangle \right)$. We prepare a gate pool with rx0, rx1, ry0, ry1, rz0, rz1, cnot01, cnot10 for the $N=2$ demo. Amongst the eight gates, six are parameterized.

In [3]:
def rx0(theta):
    return K.kron(
        K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._x_matrix, K.eye(2)
    )


def rx1(theta):
    return K.kron(
        K.eye(2), K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._x_matrix
    )


def ry0(theta):
    return K.kron(
        K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._y_matrix, K.eye(2)
    )


def ry1(theta):
    return K.kron(
        K.eye(2), K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._y_matrix
    )


def rz0(theta):
    return K.kron(
        K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._z_matrix, K.eye(2)
    )


def rz1(theta):
    return K.kron(
        K.eye(2), K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._z_matrix
    )


def cnot01():
    return K.cast(K.convert_to_tensor(tc.gates._cnot_matrix), ctype)


def cnot10():
    return K.cast(
        K.convert_to_tensor(
            np.array([[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
        ),
        ctype,
    )


ops_repr = ["rx0", "rx1", "ry0", "ry1", "rz0", "rz1", "cnot01", "cnot10"]

In [4]:
n, p, ch = 2, 3, 8
# number of qubits, number of layers, size of operation pool

target = tc.array_to_tensor(np.array([1, 0, 0, 1.0]) / np.sqrt(2.0))
# target wavefunction, we here use GHZ2 state as the objective target


def ansatz(params, structures):
    c = tc.Circuit(n)
    params = K.cast(params, ctype)
    structures = K.cast(structures, ctype)
    for i in range(p):
        c.any(
            0,
            1,
            unitary=structures[i, 0] * rx0(params[i, 0])
            + structures[i, 1] * rx1(params[i, 1])
            + structures[i, 2] * ry0(params[i, 2])
            + structures[i, 3] * ry1(params[i, 3])
            + structures[i, 4] * rz0(params[i, 4])
            + structures[i, 5] * rz1(params[i, 5])
            + structures[i, 6] * cnot01()
            + structures[i, 7] * cnot10(),
        )
    s = c.state()
    loss = K.sum(K.abs(target - s))
    return loss


vag1 = K.jit(K.vvag(ansatz, argnums=0, vectorized_argnums=1))

## Probability Ensemble Approach

This approach is more practical and experimental relevant and is the same algorithm described in Ref.1, though we here use advanced vmap to accelerate the simulation of circuits with different structures.

In [5]:
def sampling_from_structure(structures, batch=1):
    prob = K.softmax(K.real(structures), axis=-1)
    return np.array([np.random.choice(ch, p=K.numpy(prob[i])) for i in range(p)])


@K.jit
def best_from_structure(structures):
    return K.argmax(structures, axis=-1)


@K.jit
def nmf_gradient(structures, oh):
    """
    compute the Monte Carlo gradient with respect of naive mean-field probabilistic model
    """
    choice = K.argmax(oh, axis=-1)
    prob = K.softmax(K.real(structures), axis=-1)
    indices = K.transpose(K.stack([K.cast(tf.range(p), "int64"), choice]))
    prob = tf.gather_nd(prob, indices)
    prob = K.reshape(prob, [-1, 1])
    prob = K.tile(prob, [1, ch])

    return tf.tensor_scatter_nd_add(
        tf.cast(-prob, dtype=ctype),
        indices,
        tf.ones([p], dtype=ctype),
    )


nmf_gradient_vmap = K.vmap(nmf_gradient, vectorized_argnums=1)

In [6]:
verbose = False
epochs = 400
batch = 256
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.06, 100, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.12))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
nnp = K.implicit_randn(stddev=0.02, shape=[p, 6], dtype=rtype)
stp = K.implicit_randn(stddev=0.02, shape=[p, 8], dtype=rtype)
avcost1 = 0
for epoch in range(epochs):  # iteration to update strcuture param
    avcost2 = avcost1
    costl = []
    batched_stuctures = K.onehot(
        np.stack([sampling_from_structure(stp) for _ in range(batch)]), num=8
    )
    infd, gnnp = vag1(nnp, batched_stuctures)
    gs = nmf_gradient_vmap(stp, batched_stuctures)  # \nabla lnp
    gstp = [K.cast((infd[i] - avcost2), ctype) * gs[i] for i in range(infd.shape[0])]
    gstp = K.real(K.sum(gstp, axis=0) / infd.shape[0])
    avcost1 = K.sum(infd) / infd.shape[0]
    nnp = network_opt.update(gnnp, nnp)
    stp = structure_opt.update(gstp, stp)

    if epoch % 40 == 0 or epoch == epochs - 1:
        print("----------epoch %s-----------" % epoch)
        print(
            "batched average loss: ",
            np.mean(avcost1),
        )

        if verbose:
            print(
                "strcuture parameter: \n",
                stp.numpy(),
                "\n network parameter: \n",
                nnp.numpy(),
            )

        cand_preset = best_from_structure(stp)
        print("best candidates so far:", [ops_repr[i] for i in cand_preset])
        print(
            "corresponding weights for each gate:",
            [K.numpy(nnp[j, i]) if i < 6 else 0.0 for j, i in enumerate(cand_preset)],
        )

----------epoch 0-----------
batched average loss:  1.438692604002888
best candidates so far: ['cnot01', 'rx0', 'rx1']
corresponding weights for each gate: [0.0, -0.049711242696246834, 0.0456804722145847]
----------epoch 40-----------
batched average loss:  1.0024311791127296
best candidates so far: ['cnot01', 'ry0', 'cnot01']
corresponding weights for each gate: [0.0, -0.1351106165832465, 0.0]
----------epoch 80-----------
batched average loss:  0.09550699673720528
best candidates so far: ['ry0', 'ry0', 'cnot01']
corresponding weights for each gate: [-0.06370593607560585, -0.7355997299177472, 0.0]
----------epoch 120-----------
batched average loss:  0.0672150785213724
best candidates so far: ['ry0', 'ry0', 'cnot01']
corresponding weights for each gate: [-0.062430880135008346, -0.7343246757666638, 0.0]
----------epoch 160-----------
batched average loss:  0.07052086338808516
best candidates so far: ['ry0', 'ry0', 'cnot01']
corresponding weights for each gate: [-0.060554804305087445, -

## Directly Optimize the Structure Parameters

Since we are using numerical simulation anyway, we can directly optimize the structure parameter and omit whether the super circuit is unitary or not, this approach can be faster and more reliable for some scenarios.

In [7]:
def ansatz2(params, structures):
    c = tc.Circuit(n)
    params = K.cast(params, ctype)
    structures = K.softmax(structures, axis=-1)
    structures = K.cast(structures, ctype)
    for i in range(p):
        c.any(
            0,
            1,
            unitary=structures[i, 0] * rx0(params[i, 0])
            + structures[i, 1] * rx1(params[i, 1])
            + structures[i, 2] * ry0(params[i, 2])
            + structures[i, 3] * ry1(params[i, 3])
            + structures[i, 4] * rz0(params[i, 4])
            + structures[i, 5] * rz1(params[i, 5])
            + structures[i, 6] * cnot01()
            + structures[i, 7] * cnot10(),
        )
    s = c.state()
    s /= K.norm(s)
    loss = K.sum(K.abs(target - s))
    return loss


vag2 = K.jit(K.value_and_grad(ansatz2, argnums=(0, 1)))

In [8]:
verbose = True
epochs = 700
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.05, 200, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.04))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
nnp = K.implicit_randn(stddev=0.02, shape=[p, 6], dtype=rtype)
stp = K.implicit_randn(stddev=0.02, shape=[p, 8], dtype=rtype)
for epoch in range(epochs):

    infd, (gnnp, gstp) = vag2(nnp, stp)

    nnp = network_opt.update(gnnp, nnp)
    stp = structure_opt.update(gstp, stp)
    if epoch % 70 == 0 or epoch == epochs - 1:
        print("----------epoch %s-----------" % epoch)
        print(
            "batched average loss: ",
            np.mean(infd),
        )
        if verbose:
            print(
                "strcuture parameter: \n",
                stp.numpy(),
                "\n network parameter: \n",
                nnp.numpy(),
            )

        cand_preset = best_from_structure(stp)
        print("best candidates so far:", [ops_repr[i] for i in cand_preset])
        print(
            "corresponding weights for each gate:",
            [K.numpy(nnp[j, i]) if i < 6 else 0.0 for j, i in enumerate(cand_preset)],
        )

----------epoch 0-----------
batched average loss:  1.3046788213442395
strcuture parameter: 
 [[ 0.07621179  0.04934165  0.04669995  0.04737751  0.02036102  0.01170415
   0.03786593 -0.05644197]
 [ 0.01168381  0.0561013   0.02979136  0.03134415  0.03763557  0.03739249
   0.03408754 -0.05335854]
 [ 0.03540374  0.03219197  0.01680129  0.02014464  0.06939972  0.02393527
   0.04619596 -0.01844729]] 
 network parameter: 
 [[-0.0584098   0.04281717  0.0642035   0.06008445  0.0357175   0.05512457]
 [-0.07067937  0.04410743  0.03608519  0.03465959  0.02446072  0.06917318]
 [-0.01337738  0.04776898  0.04278249  0.04917169  0.0495427   0.01059102]]
best candidates so far: ['rx0', 'rx1', 'rz0']
corresponding weights for each gate: [-0.058409803714939854, 0.04410743113093344, 0.04954270315507654]
----------epoch 70-----------
batched average loss:  1.0081966098666586
strcuture parameter: 
 [[-0.91750096  0.35057522  0.32585577  0.37681816  1.77239369  1.7734987
   1.80143958 -0.38591221]
 [ 0.3008

## Final Fine-tune

For the obtained circuit layout we can further adjust the circuit weights to make the objective more close to zero.

In [9]:
chosen_structure = K.onehot(np.array([2, 4, 6]), num=8)
chosen_structure = K.reshape(chosen_structure, [1, p, ch])
chosen_structure

<tf.Tensor: shape=(1, 3, 8), dtype=float32, numpy=
array([[[0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.]]], dtype=float32)>

In [10]:
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(1e-3))
nnp = K.implicit_randn(stddev=0.02, shape=[p, 6], dtype=rtype)
verbose = True
epochs = 600
for epoch in range(epochs):
    infd, gnnp = vag1(nnp, chosen_structure)
    nnp = network_opt.update(gnnp, nnp)
    if epoch % 60 == 0 or epoch == epochs - 1:
        print(epoch, "loss: ", K.numpy(infd[0]))

0 loss:  0.9827084438054802
60 loss:  0.9449745688150044
120 loss:  0.8850948396917335
180 loss:  0.8048454837720991
240 loss:  0.706158632509899
300 loss:  0.5901794549931197
360 loss:  0.45808014651166296
420 loss:  0.3113751664914397
480 loss:  0.1520672098147883
540 loss:  0.0014714944860031312
599 loss:  0.0032763286672428237


## References

1. https://arxiv.org/pdf/2010.08561.pdf